### Let's develop a chatbot based on groq llms using langchain and deploy it on haggingface 

In [ ]:
import os 
import warnings
from langchain_groq import ChatGroq
from langchain_community.llms import HuggingFaceEndpoint

warnings.filterwarnings("ignore")

In [ ]:
groq_llm = ChatGroq(
    groq_api_key = os.environ["GROQ_API_KEY"],
    model_name = "llama-3.1-8b-instant", #Check groq website for other llm ids
    temperature = 0.6 
)

### Prompt Templates nd langchain chains

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
prompt_template = PromptTemplate(input_variables = ["word"],
                                 template = "What are 5 synonyms of word {word}")
prompt_template.format(word = "impossible")

In [ ]:
chain = LLMChain(llm = groq_llm, prompt = prompt_template)
synonyme_response = chain.run("impossible")
print (synonyme_response)

In [ ]:
print (chain.run("possible"))

### Combining multiple chains using simple sequential chain 

In [ ]:
import os 
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
from langchain.chains import SimpleSequentialChain

In [ ]:
groq_llm = ChatGroq(
    groq_api_key = os.environ["GROQ_API_KEY"],
    model_name = "llama-3.1-8b-instant",
    temperature = 0.6,
)

In [ ]:
first_prompt = PromptTemplate(input_variables =["word"], 
                             template = "What are one synonym of word {word}, return the synonyme and nothing else" )
first_chain = LLMChain(llm = groq_llm, prompt = first_prompt)

In [ ]:
first_chain.run(word = "impossible")

In [ ]:
second_prompt = PromptTemplate(input_variables = ["synonyme"],
                                template = "What is the translation of this word {synonyme} in french")
second_chain = LLMChain(llm = groq_llm, prompt = second_prompt)

In [ ]:
print(second_chain.run(synonyme = "Infeasible"))

In [ ]:
#Sequantial chain 
simple_sequence_chain = SimpleSequentialChain(chains = [first_chain, second_chain])

In [ ]:
output = sequence_chain.run("nice")
print (output)

### Sequential chains 

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain

In [ ]:
first_template = PromptTemplate(input_variables = ["proverb"], 
                                template = "Complete this proverb {proverb}, return the complete proverb only nothin more.")
first_chain = LLMChain(llm = groq_llm, prompt = first_template, output_key ="proverb_completed")

In [ ]:
print (first_chain.run("He who plays with the sword"))

In [ ]:
second_template = PromptTemplate (input_variables = ["proverb_completed"], 
                                  template = """
                                                Translate this proverbe <{proverb_completed}> to french, 
                                                return only one translation and nothing more.
                                             """)
second_chain = LLMChain(llm = groq_llm, prompt = second_template, output_key = "translation")

In [ ]:
print (second_chain.run("He who plays with the sword, must be prepared to be wounded."))

In [ ]:
sequential_chain = SequentialChain(chains = [first_chain, second_chain],
                input_variables = ["proverb"],
                output_variables = ["proverb_completed", "translation"])

In [ ]:
sequential_chain({"proverb": "He who plays with the sowrd"})

### Chat models, System messages 

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

In [ ]:
messages = [(SystemMessage("You are an expert in proverbs, user will give you one theme, you have to present 3 proverbs that corresponds to that theme")),
            (HumanMessage("Courage"))]
output = groq_llm.invoke(messages)
print(output.content)

### Message persistence

In [ ]:
import os
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph, MessagesState
from langchain_groq import ChatGroq

In [ ]:
#Model
groq_llm = ChatGroq(
    groq_api_key = os.environ["GROQ_API_KEY"],
    model_name = "llama-3.1-8b-instant", #Check groq website for other llm ids
    temperature = 0.6 
)

In [ ]:
#Worflow 
workflow = StateGraph(state_schema = MessagesState)
#define how to call the model 
def call_model(state: MessagesState):
    response = groq_llm.invoke(state["messages"])
    return {"messages": response}
#node and edges
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
#Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer = memory)

In [ ]:
#Set configuration
config_1 = {"configurable": {"thread_id": "chat_1"}}
config_2 = {"configurable": {"thread_id": "chat_2"}}

In [ ]:
def invoke_app(query : str, config):
    response = app.invoke({"messages": [HumanMessage(query)]}, config)
    return response["messages"][-1].content

In [ ]:
query_1 = "Hello, it is 4 o clock pm, here in morocco, what time could be in new york, America ?"
invoke_app(query_1, config_1)

In [ ]:
query_2 = "What would be the time in morocco in 2 hours, give the time only, nothing more."
invoke_app(query_2, config_1)

In [ ]:
invoke_app(query_2, config_2)